In [ ]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as layers
import numpy as np


In [ ]:
# ВВЕДИТЕ ПУТЬ К JSON-ДАТАСЕТУ
JSON_PATH = 'ВАШ ПУТЬ'
import json

with open(JSON_PATH, 'r', encoding='utf8') as f:
  data = json.load(f)
  f.close()


Mounted at /content/drive


In [ ]:
# Функция для докомплектации пустыми векторами матрицы поездов
def find_max_train_count(situations):
    max_train_count = 0
    full_timetable = []

    # Находим максимальное количество поездов
    for situation in situations:
        full_timetable = situation['full_timetable']
        train_count = len(full_timetable)

        if train_count > max_train_count:
            max_train_count = train_count
    return max_train_count

# Конвертер временных интервалов
def convert_time_intervals(time_intervals):
    converted_arr = []
    converted_dep = []
    for interval in time_intervals:
        start_time, end_time = interval.split(' - ')
        start_hour, start_minute = map(int, start_time.split(':'))
        end_hour, end_minute = map(int, end_time.split(':'))

        total_start_minutes = start_hour * 60 + start_minute
        total_end_minutes = end_hour * 60 + end_minute

        converted_arr.append(total_start_minutes)
        converted_dep.append(total_end_minutes)

    return converted_arr, converted_dep

# Функция для приведения данных о поездах в читаемый вид
def transform_train_data(data, max_train_count):
    transformed_data = []

    zero_matrix = np.zeros((max_train_count, 4, 7))
    # route - free_carriage - time_arr - time_dep
    for i, entry in enumerate(data.items()):
      route = list(map(int, entry[1]["route"])) + [0] * (7 - len(entry[1]["route"]))
      free_carriage = list(map(int, entry[1]['free_carriage'])) + [0] * (7 - len(entry[1]['free_carriage']))
      time_arr = convert_time_intervals(entry[1]['timetable'])[0] + [0] * (7 - len(entry[1]['timetable']))
      time_dep = convert_time_intervals(entry[1]['timetable'])[1] + [0] * (7 - len(entry[1]['timetable']))

      info_list = [route, free_carriage, time_arr, time_dep]
      for x in range(len(zero_matrix[i])):
        for y in range(len(zero_matrix[i][x])):
          zero_matrix[i][x][y] = info_list[x][y]

    return zero_matrix


max_train_count = find_max_train_count(data)

def normilize(data):
  normilized_stations_data = []
  normilized_train_data = []

  global max_train_count
  for situation in data:
      stations_data = situation['stations']
      stations = {key: value for key, value in stations_data.items()}

      N = 7
      unused_vagons_per_situation = np.zeros((N, N))

      for source_station, carriages_in_order in stations_data.items():
          source_station_index = list(stations_data.keys()).index(source_station)
          for carriage_destination, num_carriages in enumerate(carriages_in_order):
              destination_station_index = carriage_destination
              if source_station_index != destination_station_index:
                  unused_vagons_per_situation[source_station_index][destination_station_index] = num_carriages

      normilized_stations_data.append(unused_vagons_per_situation)

      trains_data = situation['full_timetable']

      sit_norm_train_data = transform_train_data(trains_data, max_train_count)
      normilized_train_data.append(sit_norm_train_data)


  train_pack = [normilized_stations_data[:90000], normilized_train_data[:90000]]
  test_pack = [normilized_stations_data[90001:], normilized_train_data[90001:]]

  return train_pack, test_pack


In [ ]:
# Подготовка к реализации нейросети

train_pack, test_pack = normilize(data)
del data

In [ ]:
class GraphNeuralNetwork(tfk.Model):
    def __init__(self, num_stations=7, num_train_vectors=43):
        super(GraphNeuralNetwork, self).__init__()
        self.num_stations = num_stations
        self.num_train_vectors = num_train_vectors
        self.shape_output = (self.num_train_vectors, 7, 7)

        self.input_train_vectors = layers.Input(shape=(self.num_train_vectors, 4, 7), name='input_train_vectors')
        # self.create_mask()

        self.model = self.create_model()

    # def create_mask(self):
    #     self.mask = np.ones_like(self.input_train_vectors, dtype=np.int32)
    #     for i in range(self.input_train_vectors.shape[0]):
    #       for j in range(self.input_train_vectors.shape[1]):
    #           for k in range(self.input_train_vectors.shape[2]):
    #               if self.input_train_vectors[i, j, k] == 0:
    #                   self.mask[i, j, k] = 0
    #                   self.mask = tf.constant(self.mask, dtype=tf.float32)

    def custom_loss(initial_matrix):
      def loss(y_true, y_pred):
          return tfk.backend.sum(tfk.backend.abs(initial_matrix - y_pred))

      return loss

    def create_model(self):
        reshaped_input_train_vectors = layers.Reshape((self.num_train_vectors * 4, 7), name='preLSTM_reshape')(self.input_train_vectors)

        lstm_layer = layers.LSTM(16, return_sequences=True)(reshaped_input_train_vectors)
        flat = layers.Flatten()(lstm_layer)
        dense_layer = layers.Dense(16, activation="relu")(flat)

        hidden_layer1 = layers.Dense(32, activation="relu")(dense_layer)
        hidden_layer2 = layers.Dense(16, activation="relu")(hidden_layer1)


        output_layer_prereshape = layers.Dense(self.num_train_vectors * 7 * 7, activation="softmax", name="output")(hidden_layer2)
        output_layer = layers.Reshape(self.shape_output, name='final_reshape')(output_layer_prereshape)

        return tfk.Model(inputs=[self.input_train_vectors], outputs=output_layer)

model = GraphNeuralNetwork(7, 49)

In [ ]:
########################################################################################
# Реализация матмодели. Из-за специфики задания пришлось отказаться в пользу нейросети #
########################################################################################


# from scipy.optimize import linprog

# from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus

# initial_matrix = unused_vagons_per_situation

# # Оптимизационная задача
# model = LpProblem(name="railway_optimization", sense=LpMinimize)

# # Переменные решения
# var = [[[LpVariable(name=f"var_{i}_{j}_{k}", lowBound=0, cat="Integer") for k in range(7)] for j in range(7)] for i in range(len(trains_data))]

# # Определяю целевую функцию (минимизация суммарной разницы)
# model += lpSum(initial_matrix[j][k] - var[i][j][k] for i in range(len(trains_data)) for j in range(7) for k in range(7))

# # Добавляем ограничения на количество вагонов на каждой станции
# for train_index, trains in trains_data.items():
#     for train in trains:
#         model += lpSum(var[int(train_index)][1][1] for j in range(7)) <= train[int(train_index)]["available_carriages"]

# # Добавляем ограничения на перевозку вагонов по маршруту каждого поезда
# for train_index, train in trains_data.items():
#     for j in range(len(train["route"]) - 1):
#         from_station = train["route"][j] - 1
#         to_station = train["route"][j + 1] - 1
#         model += lpSum(var[train_index][from_station][to_station]) == train["available_carriages"][from_station]

# # Решаем оптимизационную задачу
# model.solve()

# # Выводим результаты
# if LpStatus[model.status] == "Optimal":
#     print("Оптимальное распределение вагонов:")
#     for i in range(len(trains_data)):
#         for j in range(7):
#             for k in range(7):
#                 print(f"Поезд {i+1}, станция {j+1} -> {k+1}: {var[i][j][k].value()}")
# else:
#     print("Задача не имеет оптимального решения.")
